In [1]:
import pygsheets
import pandas as pd
import numpy as np
import itertools
import os
from dotenv import load_dotenv
import praw
import datetime as dt

import spacy

Get reddit instance for PRAW

In [2]:
load_dotenv()
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")
username = os.getenv("user")
password = os.getenv("password")
user_agent = os.getenv("user_agent")

In [3]:
# creating an authorized reddit instance
def login(client_id, client_secret, username, password, user_agent):
    reddit = praw.Reddit(client_id = client_id,
                     client_secret = client_secret,
                     username = username,
                     password = password,
                     user_agent = user_agent)
    return reddit

In [4]:
reddit = login(client_id, client_secret, username, password, user_agent)

Get google sheets instance for pygsheets

In [5]:
secretsPath = r"client_secret_623798116104-13e1be1fv1ucarscmae78mdvftdbu2tg.apps.googleusercontent.com.json"
gc = pygsheets.authorize(outh_file=secretsPath)

Open google sheet/worksheet

In [6]:
sh = gc.open("pySheets")
wks = sh.worksheet_by_title("postInfo")

In [27]:
def updatePosts(subList=['test'], timeFilter='week', setLimit=100, wksToUse=wks, rInstance=reddit): # dfToUse=df if we decide to put df back in here
    """
    Function to update the google worksheet we are using.  
    This function goes through each subreddit in subList individually so as to get top posts for each subreddit rather than their combined subreddit object
    
    Args:
        subList (list): list of all subreddits to iterate over
        timeFilter (str): length of time to look through top posts.  Possible options of 'hour', 'day', 'week', 'month'
        setLimit (int): number of top posts to look at within the timeframe
        dfToUse (dataframe): the dataframe we will put our submission data into for further use
        wksToUse (pygsheets.worksheet.Worksheet): the worksheet we'll be updating
        rInstance (praw.reddit.Reddit): the reddit instance we'll be using to extract posts
    """
    for subr in subList:
        print(f"Starting {subr}")
        subrInstance = rInstance.subreddit(subr)
        firstCellCheck = wksToUse.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')  
        currentIDs = [r[6] for i, r in enumerate(firstCellCheck[1:])]  # Get list of all current post IDs in our sheet
        for submission in subrInstance.top(time_filter=timeFilter, limit=setLimit):
            if submission.id in currentIDs:  # Don't repeat posts!
                continue
            else:
                d = {}
                d['timeCreated'] = dt.datetime.fromtimestamp(submission.created_utc)
                d['subreddit'] = submission.subreddit.display_name
                d['title'] = submission.title
                d['score'] = submission.score
                d['author'] = submission.author.name
                d['selftext'] = submission.selftext
                d['id'] = submission.id
                # dfToUse = pd.concat([dfToUse,pd.DataFrame.from_dict(d, orient='index').T], ignore_index=True, axis=0)
                
                cells = wksToUse.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')  
                lastrow = len(cells)  # Must update every time so as to properly append rows
                
                # make the timeCreated json serializable for appending to worksheet
                dvals = list(d.values())
                dvals[0] = dvals[0].isoformat()
                
                
                wksToUse.insert_rows(lastrow, number=1, values=dvals)
            
        print(f"{subr} complete")
        
    print("All subreddits complete")

In [28]:
testvals = wks.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')  
currentIDs = [r[6] for i, r in enumerate(testvals[1:])]
if '123rfgy' in currentIDs:
    print("yes")

yes


In [29]:
updatePosts(subList=['news', 'nottheonion', 'inthenews', 'offbeat'], setLimit=100)  # Lets test with some google-suggested news subreddits

Starting news
news complete
Starting nottheonion
nottheonion complete
Starting inthenews
inthenews complete
Starting offbeat
offbeat complete
All subreddits complete


In [9]:
def cleanSheetByTime(clientToUse=gc, sheetToUse=sh, wksToUse=wks):
    """
    Function to clean a given worksheet by the timeCreated column.  This function uses the clientToUse 
    pygsheets client to look at the wksToUse worksheet within the sheetToUse spreadsheet.
    It then removes any rows where the timeCreated column shows a post is from more than a week ago.
    
    Args:
        clientToUse (pygsheets.client.Client): the pygsheets client we'll be using
        sheetToUse (pygsheets.spreadsheet.Spreadsheet): the pygsheets spreadsheet we'll be cleaning in
        wksToUse (pygsheets.worksheet.Worksheet): the worksheet we'll be cleaning
    """
    cells = wksToUse.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')  
    timeNow = dt.datetime.now()
    cutoffTime = timeNow - dt.timedelta(weeks=1)  # Set the cutoff time
    # rowsToDelete = [i+2 for i, r in enumerate(cells[1:]) if (dt.datetime.strptime(r[0], "%Y-%m-%d %H:%M:%S")  <=  dt.datetime.strptime("2023-03-27 0:17:11", "%Y-%m-%d %H:%M:%S"))] # For testing specific time
    rowsToDelete = [i+2 for i, r in enumerate(cells[1:]) if (dt.datetime.strptime(r[0], "%Y-%m-%d %H:%M:%S")  <=  cutoffTime)]  # i+2 as 1 for removing 1st row of cells and another for sheets counting from 1 instead of 0

    # reqs just tells batch_update which rows to delete using the API wrapper
    reqs = [
    {
        "deleteDimension": {
            "range": {
                "sheetId": wksToUse.id,
                "startIndex": e-1,
                "endIndex": e,
                "dimension": "ROWS",
            }
        }
    }
    for e in rowsToDelete
    ]
    reqs.reverse()
    clientToUse.sheet.batch_update(sheetToUse.id, reqs)

In [360]:
cleanSheetByTime()

In [17]:
df = wks.get_as_df()
df.head()

,timeCreated,subreddit,title,score,author,selftext,id
0,2023-03-25 7:39:22,funny,Ever wondered what the opposite of a stork was?,134109,The_Human_Event,,121knvf
1,2023-03-23 9:59:09,funny,Today a zebra escaped a zoo in Korea and this ...,121769,lama22,,11zmyjg
2,2023-03-22 17:10:32,funny,I spotted this flyer on 9th Ave in NYC and I h...,81813,aetryx,,11yym3k
3,2023-03-24 15:44:05,funny,Yea... The tape is staying,77474,Epidemiology2,,120xylm
4,2023-03-28 8:05:28,funny,"Car for sale, like new",68634,flyingorange,,124o55z


In [15]:
nerWks = sh.worksheet_by_title("nerInfo")

In [34]:
def getNER(sourceWks=wks, nerWks=nerWks ):

    df = sourceWks.get_as_df()  # Get the postInfo worksheet as a dataframe
    
    firstCellCheck = nerWks.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')  
    currentIDs = [r[0] for i, r in enumerate(firstCellCheck[1:])]  # Get list of all current post IDs in our sheet
    
    NER = spacy.load("en_core_web_sm")
    print(f"There are {len(df)} rows in the source worksheet")
    for i, row in df.iterrows():
        if i % 50 == 0:
            print(f"{i}th row completed")
        if row['id'] in currentIDs:  # Don't repeat work!
            continue
        else:
            d = {}
            d['postID'] = row['id']
            nerText = NER(row['title'] + " " + row['selftext'])
            for word in nerText.ents:
                d['nerWord'] = word.text
                d['nerLabel'] = word.label_
                cells = nerWks.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')  
                lastrow = len(cells)
                dvals = list(d.values())
                nerWks.insert_rows(lastrow, number=1, values=dvals)
    print("Done!")

In [35]:
getNER()

There are 364 rows in the source worksheet
0th row completed
50th row completed
100th row completed
150th row completed
200th row completed
250th row completed
300th row completed
350th row completed
Done!


In [66]:
df2 = nerWks.get_as_df()
df2.head()

,postID,postSelfText,postTitle,nerWord,nerLabel
0,11zmyjg,,Today a zebra escaped a zoo in Korea and this ...,Today,DATE
1,11zmyjg,,Today a zebra escaped a zoo in Korea and this ...,Korea,GPE
2,11yym3k,,I spotted this flyer on 9th Ave in NYC and I h...,9th Ave,WORK_OF_ART
3,11yym3k,,I spotted this flyer on 9th Ave in NYC and I h...,NYC,LOC
4,11zmpjf,,If you could place any object on the surface o...,Mars,LOC
